In [1]:
#| default_exp sandbox

In [2]:
#| hide
from nbdev.showdoc import *

In [3]:
#| hide
import nbdev; nbdev.nbdev_export()

In [4]:
#| hide
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [5]:
from dotenv import load_dotenv
import os
load_dotenv("../.env")

True

In [6]:
os.environ['HTTP_PROXY'] = "http://127.0.0.1:20171"
os.environ['HTTPS_PROXY'] = "http://127.0.0.1:20171"
os.environ['http_proxy'] = "http://127.0.0.1:20171"
os.environ['https_proxy'] = "http://127.0.0.1:20171"

In [7]:
os.getenv('GC_QUOTE_API_CREDENTIALS')
os.getenv('HTTP_PROXY')
os.getenv('HTTPS_PROXY')

'res/quote-api-365206-39d39b2fde07.json'

'http://127.0.0.1:20171'

'http://127.0.0.1:20171'

In [8]:
from dotenv import dotenv_values
config = dotenv_values("../.env")
key = config.get('GC_QUOTE_API_CREDENTIALS')

In [9]:
from google.cloud import bigquery
import pandas_gbq
from google.oauth2 import service_account
credentials = service_account.Credentials.from_service_account_file("../" + key)

In [10]:
client = bigquery.Client(credentials=credentials)

QUERY = """
		select *
		from `quote-api-365206.report.ride_dispatch_v` r
		where dispatch_amount_net_usd is not null
        limit 200
		"""

In [11]:
QUERY_DISINCT = """
		select count (*)
		from `quote-api-365206.report.ride_dispatch_v` r
		where trip_type is null
        limit 200
		"""

In [12]:
!echo $HTTP_PROXY

http://127.0.0.1:20171


In [13]:
#query_job = client.query(QUERY)

In [14]:

df = pandas_gbq.read_gbq(QUERY,credentials=credentials)

Downloading: 100%|██████████|


In [15]:

df_trip_type = pandas_gbq.read_gbq(QUERY_DISINCT,credentials=credentials)

Downloading: 100%|██████████|


In [16]:

df.head()

,ride_id,trip_no,dispatch_id,trip_type,ride_stat,dispatch_stat,partner_tran_id,partner_id,from_fleet_id_as_customer,pickup_date,...,has_complaint,has_fleet_complaint,has_penalty_complaint,has_ops_complaint,gps_ad_check,gps_cns_check,score,incoming_dispatch_info_last_updated_at,dispatch_info_last_updated_at,etl_time
0,2429723,0,583182,None,Accepted,At destination,268620,1,16,2023-02-03,...,1,0,1,1,0,0,None,2023-03-31 23:22:47+00:00,2023-03-31 23:22:47+00:00,2024-09-17 07:14:54.748547+00:00
1,2324292,0,345056,None,Accepted,At destination,161955,1,16,2022-07-28,...,0,0,0,0,0,0,None,2022-07-08 20:18:48+00:00,2022-07-28 15:37:37+00:00,2024-09-17 07:14:54.748547+00:00
2,3531231,1,2800358,point2point,Cancelled,Accepted,1377216,1,16,2024-06-30,...,0,0,0,0,0,0,None,2024-06-13 13:12:21+00:00,2024-06-13 14:10:50+00:00,2024-09-17 07:14:54.748547+00:00
3,2443954,0,609438,None,Accepted,Accepted,283048,1,16,2023-02-18,...,1,0,1,1,0,0,4.000000000,2023-02-25 01:40:42+00:00,2023-02-25 01:40:42+00:00,2024-09-17 07:14:54.748547+00:00
4,2328037,0,354302,None,Accepted,At destination,165711,1,16,2022-10-07,...,0,0,0,0,0,0,None,2022-11-02 05:51:37+00:00,2022-10-07 15:56:36+00:00,2024-09-17 07:14:54.748547+00:00


In [17]:
QUERY_ROUTE = """
        SELECT r.*, p1.lat from_lat, p1.lng from_lng, p2.lat to_lat, p2.lng to_lng
        FROM `report.ride_dispatch_v` r
        left JOIN `report.dim_place` p1
        ON r.pickup_place_id=p1.id
        INNER JOIN `report.dim_place` p2
        ON r.pickup_place_id=p2.id
        ORDER BY ride_id DESC
        limit 2;
		"""


In [18]:
df_route = pandas_gbq.read_gbq(QUERY_ROUTE,credentials=credentials)

Downloading: 100%|██████████|


In [19]:
df_route.head()

,ride_id,trip_no,dispatch_id,trip_type,ride_stat,dispatch_stat,partner_tran_id,partner_id,from_fleet_id_as_customer,pickup_date,...,gps_ad_check,gps_cns_check,score,incoming_dispatch_info_last_updated_at,dispatch_info_last_updated_at,etl_time,from_lat,from_lng,to_lat,to_lng
0,3820538,0,3330003,point2point,Pending,None,1639243,2,18,2025-01-14,...,0,0,None,2024-09-17 07:02:01+00:00,NaT,2024-09-17 07:14:54.748547+00:00,-33.972900000,18.602380000,-33.972900000,18.602380000
1,3820537,0,3329998,point2point,Pending,None,1639242,2692,57395,2024-09-28,...,0,0,None,2024-09-17 07:01:16+00:00,NaT,2024-09-17 07:14:54.748547+00:00,45.653276000,12.202081000,45.653276000,12.202081000


In [20]:
df_route.dtypes

ride_id                                                 Int64
trip_no                                                 Int64
dispatch_id                                             Int64
trip_type                                              object
ride_stat                                              object
dispatch_stat                                          object
partner_tran_id                                         Int64
partner_id                                              Int64
from_fleet_id_as_customer                               Int64
pickup_date                                            dbdate
pickup_time                                            dbtime
pickup_datetime                           datetime64[us, UTC]
pickup_local_datetime                          datetime64[us]
pickup_local_datetime_str                              object
pickup_place_id                                         Int64
dropoff_place_id                                        Int64
passenge

In [21]:
import pandas as pd
import pandera as pa

# data to validate
df = pd.DataFrame({
    "column1": [1, 4, 0, 10, 9, 11],
    "column2": [-1.3, -1.4, -2.9, -10.1, -20.4, -2.0],
    "column3": ["value_1", "value_2", "value_3", "value_2", "value_1", "value_2"],
})

# define schema
schema = pa.DataFrameSchema({
    "column1": pa.Column(int, checks=pa.Check.le(10)),
    "column2": pa.Column(float, checks=pa.Check.lt(-1.2)),
    "column3": pa.Column(str, checks=[
        pa.Check.str_startswith("value_"),
        # define custom checks as functions that take a series as input and
        # outputs a boolean or boolean Series
        pa.Check(lambda s: s.str.split("_", expand=True).shape[1] == 2)
    ]),
})

validated_df = schema(df)
print(validated_df)

SchemaError: Column 'column1' failed element-wise validator number 0: less_than_or_equal_to(10) failure cases: 11

In [3]:
from pandera.typing import Series

class Schema(pa.DataFrameModel):

    column1: int = pa.Field(le=10)
    column2: float = pa.Field(lt=-1.2)
    column3: str = pa.Field(str_startswith="value_")

    @pa.check("column3")
    def column_3_check(cls, series: Series[str]) -> Series[bool]:
        """Check that column3 values have two elements after being split with '_'"""
        return series.str.split("_", expand=True).shape[1] == 2

Schema.validate(df)

SchemaError: Column 'column1' failed element-wise validator number 0: less_than_or_equal_to(10) failure cases: 11

In [22]:
from datetime import datetime

def validate_datetime(date_string, format_string):
    """
    Validates if a string is a valid datetime according to the given format.
    """

    try:
        datetime.strptime(date_string, format_string)
        return True
    except ValueError:
        return False
    
def validate_datetime_in_iso_format(date_text):
        try:
            datetime.fromisoformat(date_text)
            return True
        except ValueError:
            # raise ValueError("Incorrect data format, should be YYYY-MM-DD")
            return False
# Example usage
date_strings = [
    "2023-12-25 24:00:00",
    "2023-12-25 12:10:00",
    "2023-12-25 23:00:00",
    "2011-11-04",
    "20111104",
    "2011-11-04T00:05:23",
    "2011-11-04T00:05:23Z",
    "2011-11-04T00:05:23.283185",
    "2011-11-04T00:05:23.283185+08:00",
    "20111104T000523",
    "20111104T000523.283185",
    "2011-11-04T00:05:23+08:00",
    ]
format_string = "%Y-%m-%d"

for s in date_strings:
    if validate_datetime_in_iso_format(s):
        print(f"{s} is Valid datetime string")
    else:
        print(f"{s} is Invalid datetime string")

2023-12-25 24:00:00 is Invalid datetime string
2023-12-25 12:10:00 is Valid datetime string
2023-12-25 23:00:00 is Valid datetime string
2011-11-04 is Valid datetime string
20111104 is Valid datetime string
2011-11-04T00:05:23 is Valid datetime string
2011-11-04T00:05:23Z is Valid datetime string
2011-11-04T00:05:23.283185 is Valid datetime string
2011-11-04T00:05:23.283185+08:00 is Valid datetime string
20111104T000523 is Valid datetime string
20111104T000523.283185 is Valid datetime string
2011-11-04T00:05:23+08:00 is Valid datetime string


In [23]:
df_datetime_str = pd.DataFrame(date_strings)
df_datetime_str.columns = ['datetime']
# df_datetime_str
df_datetime_str['validity'] = df_datetime_str.apply(lambda x: validate_datetime_in_iso_format(x.iloc[0]), axis=1)
df_datetime_str


,datetime,validity
0,2023-12-25 24:00:00,False
1,2023-12-25 12:10:00,True
2,2023-12-25 23:00:00,True
3,2011-11-04,True
4,20111104,True
5,2011-11-04T00:05:23,True
6,2011-11-04T00:05:23Z,True
7,2011-11-04T00:05:23.283185,True
8,2011-11-04T00:05:23.283185+08:00,True
9,20111104T000523,True


In [24]:
df_datetime = df_datetime_str[df_datetime_str.apply(lambda x: validate_datetime_in_iso_format(x.iloc[0]), axis=1)]
df_datetime


,datetime,validity
1,2023-12-25 12:10:00,True
2,2023-12-25 23:00:00,True
3,2011-11-04,True
4,20111104,True
5,2011-11-04T00:05:23,True
6,2011-11-04T00:05:23Z,True
7,2011-11-04T00:05:23.283185,True
8,2011-11-04T00:05:23.283185+08:00,True
9,20111104T000523,True
10,20111104T000523.283185,True


In [25]:
df_datetime['datetime']

1                  2023-12-25 12:10:00
2                  2023-12-25 23:00:00
3                           2011-11-04
4                             20111104
5                  2011-11-04T00:05:23
6                 2011-11-04T00:05:23Z
7           2011-11-04T00:05:23.283185
8     2011-11-04T00:05:23.283185+08:00
9                      20111104T000523
10              20111104T000523.283185
11           2011-11-04T00:05:23+08:00
Name: datetime, dtype: object

In [26]:
# df_datetime_str.apply(lambda x: validate_datetime_in_iso_format(x.iloc[0]), axis=1)
df_datetime1 = df_datetime.apply(lambda x: datetime.fromisoformat(x['datetime']), axis=1)
df_datetime1.name = 'datetime'
df_datetime1


1                  2023-12-25 12:10:00
2                  2023-12-25 23:00:00
3                  2011-11-04 00:00:00
4                  2011-11-04 00:00:00
5                  2011-11-04 00:05:23
6            2011-11-04 00:05:23+00:00
7           2011-11-04 00:05:23.283185
8     2011-11-04 00:05:23.283185+08:00
9                  2011-11-04 00:05:23
10          2011-11-04 00:05:23.283185
11           2011-11-04 00:05:23+08:00
Name: datetime, dtype: object

In [31]:
pd.to_datetime(df_datetime1[5])

Timestamp('2011-11-04 00:05:23')

In [32]:
import pytz
def get_timezone_abbreviation(timezone_name):
    timezone = pytz.timezone(timezone_name)
    now = datetime.now(timezone)
    return now.strftime("%Z")

def validate_timezone_in_iana(timezone_name):
     return timezone_name in pytz.all_timezones

time_zone_strings = [
    'Asia/Shanghai',
    'Asia/Mumbai',
    'America/New_York',
    'Europe/London', 
    'Eastern Standard Time', 
    'US/Eastern', 
    'America/Los_Angeles', 
    'Asia/Kolkata', 
    'Europe/London'
]

In [33]:
df_timezone_str = pd.DataFrame(time_zone_strings)
df_timezone_str.columns = ['timezone']
# df_datetime_str
df_timezone_str['validity'] = df_timezone_str.apply(lambda x: validate_timezone_in_iana(x.iloc[0]), axis=1)
df_timezone_str


,timezone,validity
0,Asia/Shanghai,True
1,Asia/Mumbai,False
2,America/New_York,True
3,Europe/London,True
4,Eastern Standard Time,False
5,US/Eastern,True
6,America/Los_Angeles,True
7,Asia/Kolkata,True
8,Europe/London,True


In [34]:

df_timezone = df_timezone_str[df_timezone_str.apply(lambda x: validate_timezone_in_iana(x.iloc[0]), axis=1)]
df_timezone


,timezone,validity
0,Asia/Shanghai,True
2,America/New_York,True
3,Europe/London,True
5,US/Eastern,True
6,America/Los_Angeles,True
7,Asia/Kolkata,True
8,Europe/London,True


In [35]:
df_abbr = df_timezone.apply(lambda x: get_timezone_abbreviation(x.loc['timezone']), axis=1)
df_abbr.name = 'abbr'
df_abbr

0    CST
2    EDT
3    BST
5    EDT
6    PDT
7    IST
8    BST
Name: abbr, dtype: object

In [36]:
df_timezone

,timezone,validity
0,Asia/Shanghai,True
2,America/New_York,True
3,Europe/London,True
5,US/Eastern,True
6,America/Los_Angeles,True
7,Asia/Kolkata,True
8,Europe/London,True


In [37]:
# df_timezone_str['validity'] = df_timezone_str.apply(lambda x: validate_timezone_in_iana(x.iloc[0]), axis=1)
# df_timezone['abbr'] = df_timezone.apply(lambda x: get_timezone_abbreviation(x.iloc[0]), axis=1)
# df_timezone.apply(lambda x: get_timezone_abbreviation(x.iloc[0]), axis=1)
df_timezone.loc[:,'abbr'] = df_abbr
df_timezone
# df_timezone


/tmp/ipykernel_464126/540448849.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_timezone.loc[:,'abbr'] = df_abbr


,timezone,validity,abbr
0,Asia/Shanghai,True,CST
2,America/New_York,True,EDT
3,Europe/London,True,BST
5,US/Eastern,True,EDT
6,America/Los_Angeles,True,PDT
7,Asia/Kolkata,True,IST
8,Europe/London,True,BST


In [38]:

df_utc_offset = (df_timezone.apply(lambda x: datetime.now(pytz.timezone(x.loc['timezone'])), axis=1)
                            .apply(lambda x: x.strftime('%z')))
df_utc_offset.name = 'utc_offset'
df_utc_offset


0    +0800
2    -0400
3    +0100
5    -0400
6    -0700
7    +0530
8    +0100
Name: utc_offset, dtype: object

In [39]:

df_timezone.loc[:,'utc_offset'] = df_utc_offset
df_timezone


/tmp/ipykernel_464126/3792723555.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_timezone.loc[:,'utc_offset'] = df_utc_offset


,timezone,validity,abbr,utc_offset
0,Asia/Shanghai,True,CST,+0800
2,America/New_York,True,EDT,-0400
3,Europe/London,True,BST,+0100
5,US/Eastern,True,EDT,-0400
6,America/Los_Angeles,True,PDT,-0700
7,Asia/Kolkata,True,IST,+0530
8,Europe/London,True,BST,+0100


In [40]:
for tz in df_timezone.loc[:,'timezone']:
    print(f"{tz}: {datetime.now(pytz.timezone(tz)).strftime('%Z')}, {datetime.now(pytz.timezone(tz)).strftime('%z')}")

Asia/Shanghai: CST, +0800
America/New_York: EDT, -0400
Europe/London: BST, +0100
US/Eastern: EDT, -0400
America/Los_Angeles: PDT, -0700
Asia/Kolkata: IST, +0530
Europe/London: BST, +0100


In [43]:
# Create a timezone object for Eastern Time
eastern = pytz.timezone('US/Eastern')

# Create a datetime object for a specific date and time in Eastern Time
dt = eastern.localize(datetime(2024, 3, 10, 2, 0)) 

# Check if DST is in effect for this datetime
print(dt.dst())

0:00:00


In [56]:
from zoneinfo import ZoneInfo

In [42]:

# Convert the datetime to UTC
dt_utc = dt.astimezone(pytz.utc)
print(dt_utc)

0:00:00
2024-03-10 07:00:00+00:00


In [55]:

dt1 = eastern.localize(datetime(2024, 3, 10, 1, 59), is_dst=False)
print(dt1, dt1.dst())

# After DST transition
dt2 = eastern.localize(datetime(2024, 3, 10, 3, 0), is_dst=False)
print(dt2, dt2.dst())


2024-03-10 01:59:00-05:00 0:00:00
2024-03-10 03:00:00-04:00 1:00:00


In [54]:
# Before DST transition
dt1 = eastern.localize(datetime(2024, 3, 10, 1, 59), is_dst=True)
print(dt1, dt1.dst())

# After DST transition
dt2 = eastern.localize(datetime(2024, 3, 10, 3, 0), is_dst=True)
print(dt2, dt2.dst())
# dt2.is_dst()

2024-03-10 01:59:00-05:00 0:00:00
2024-03-10 03:00:00-04:00 1:00:00
